<a href="https://colab.research.google.com/github/dkay7223/Sentiment_analyzer_model/blob/main/sentiment_analysis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow transformers


In [ ]:
import tensorflow as tf
from tensorflow.keras import activations, optimizers, losses
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification


import pandas as pd
import numpy as np


from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/amazon_alexa.tsv',sep='\t')

In [ ]:
df.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [ ]:
df.shape

(3150, 5)

In [ ]:
df['feedback'].value_counts()


1    2893
0     257
Name: feedback, dtype: int64

1 indicates positive review whereas 0 indicates negative review.

As we can see, our dataset has more positive reviews than negative reviews.

We have to address this class imbalance before we train our model.

We do this by sub sampling the positive reviews class to select the same number of positive reviews as the negative reviews.

In [ ]:
df[df['feedback']==0].shape

(257, 5)

In [ ]:
df[df['feedback']==1].shape

(2893, 5)

In [ ]:
df_negative=df[df['feedback']==0]
df_positive=df[df['feedback']==1]

In [ ]:
df_positive_downsampled=df_positive.sample(df_negative.shape[0])
df_positive_downsampled.shape

(257, 5)

In [ ]:
df_balanced=pd.concat([df_positive_downsampled,df_negative],axis=0)
df_balanced.shape

(514, 5)

In [ ]:
df_balanced['review_length']=df_balanced['verified_reviews'].apply(lambda x: len(x.split()))

In [ ]:
df_balanced.shape

(514, 6)

In [ ]:
x=df_balanced['verified_reviews'].to_list()


In [ ]:
y=df_balanced['feedback'].to_list()

In [ ]:
MODEL_NAME = 'distilbert-base-uncased'
# hyper parameter
MAX_LEN = 50

review = "I liked the service a lot! Keep it up!"

tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

inputs = tokenizer(review, max_length=MAX_LEN, truncation=True,padding='max_length')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
tokenized_words=tokenizer.convert_ids_to_tokens(inputs["input_ids"])
print(tokenized_words)

['[CLS]', 'i', 'liked', 'the', 'service', 'a', 'lot', '!', 'keep', 'it', 'up', '!', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [ ]:
def construct_encodings(x, tokenizer, max_len, trucation=True, padding=True):
    return tokenizer(x, max_length=max_len, truncation=trucation, padding=padding)

encodings = construct_encodings(x, tokenizer, max_len=MAX_LEN)

In [ ]:
def construct_tfdataset(encodings, y=None):
    if y:
        return tf.data.Dataset.from_tensor_slices((dict(encodings),y))
    else:
        return tf.data.Dataset.from_tensor_slices(dict(encodings))

In [ ]:
tfdataset = construct_tfdataset(encodings, y)

In [ ]:
TEST_SPLIT = 0.2
VAL_SPLIT=0.2
BATCH_SIZE = 32


train_size = int(len(x) * (1-TEST_SPLIT))
val_size=int(len(x) * (VAL_SPLIT))


In [ ]:
tfdataset = tfdataset.shuffle(len(x))

tfdataset_test = tfdataset.skip(train_size)


tfdataset_rest = tfdataset.take(train_size)


tfdataset_train=tfdataset_rest.skip(val_size)
tfdataset_val=tfdataset_rest.take(val_size)


In [ ]:
tfdataset_train = tfdataset_train.batch(BATCH_SIZE)
tfdataset_val = tfdataset_val.batch(BATCH_SIZE)
tfdataset_test = tfdataset_test.batch(BATCH_SIZE)

In [ ]:
# hyper parameter
N_EPOCHS = 10

MODEL_NAME = 'distilbert-base-uncased'

model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
optimizer = optimizers.Adam(learning_rate=3e-5)
loss = losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history=model.fit(tfdataset_train, batch_size=BATCH_SIZE, epochs=N_EPOCHS,validation_data=tfdataset_val,verbose=1)

Epoch 1/10
10/10 [==============================] - 61s 550ms/step - loss: 0.6867 - accuracy: 0.5437 - val_loss: 0.6449 - val_accuracy: 0.7255
Epoch 2/10
10/10 [==============================] - 2s 244ms/step - loss: 0.5768 - accuracy: 0.7670 - val_loss: 0.4502 - val_accuracy: 0.8725
Epoch 3/10
10/10 [==============================] - 3s 277ms/step - loss: 0.3907 - accuracy: 0.8641 - val_loss: 0.2416 - val_accuracy: 0.9412
Epoch 4/10
10/10 [==============================] - 2s 187ms/step - loss: 0.2073 - accuracy: 0.9515 - val_loss: 0.2230 - val_accuracy: 0.9216
Epoch 5/10
10/10 [==============================] - 2s 187ms/step - loss: 0.1698 - accuracy: 0.9547 - val_loss: 0.1638 - val_accuracy: 0.9412
Epoch 6/10
10/10 [==============================] - 2s 188ms/step - loss: 0.1312 - accuracy: 0.9644 - val_loss: 0.1381 - val_accuracy: 0.9706
Epoch 7/10
10/10 [==============================] - 2s 187ms/step - loss: 0.1380 - accuracy: 0.9515 - val_loss: 0.0591 - val_accuracy: 0.9804
Epoch

In [ ]:
benchmarks = model.evaluate(tfdataset_test, return_dict=True, batch_size=BATCH_SIZE,verbose=1)

4/4 [==============================] - 0s 48ms/step - loss: 0.0307 - accuracy: 0.9903


In [ ]:
def create_predictor(model, model_name, max_len):
  tokenizer = DistilBertTokenizer.from_pretrained(model_name)
  def predict_proba(text):
      x = [text]

      encodings = construct_encodings(x, tokenizer, max_len=max_len)
      tfdataset = construct_tfdataset(encodings)
      tfdataset = tfdataset.batch(1)

      preds = model.predict(tfdataset).logits
      preds = activations.softmax(tf.convert_to_tensor(preds)).numpy()

      return preds[0][1]

  return predict_proba

In [ ]:
clf = create_predictor(model, MODEL_NAME, MAX_LEN)
print(clf("The technician was very talented and polite. He completed his work quickly. I loved his work!"))

1/1 [==============================] - 1s 1s/step
0.9925504


In [ ]:
import pickle
model.save_pretrained('/content/drive/MyDrive/sentiment_analyzer_model')
with open('/content/drive/MyDrive/sentiment_analyzer_model/info.pkl', 'wb') as f:
    pickle.dump((MODEL_NAME, MAX_LEN), f)

In [ ]:
#from google.colab import files
#files.download('/content/drive/MyDrive/sentiment_analyzer_model/tf_model.h5')

In [ ]:
pip install flask_cors

In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS

In [ ]:
app = Flask(__name__)
CORS(app)
@app.route('/api/predict', methods=['POST'])
def predict():
  data = request.get_json()
  input_data = np.array(data['review'])
  predictProb = input_data
  return jsonify({"Review Probability": predictProb})

In [ ]:
if __name__ == '__main__':
  app.run(debug=True)

In [ ]:
from math import radians, sin, cos, sqrt, atan2
# Code for calculating distance between two users.
def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Radius of the Earth in kilometers (mean value)
    earth_radius = 6371.0

    # Calculate the distance
    distance = earth_radius * c

    return distance

# Example usage
user1_coordinates = (36.0, -119.000)  # New York City
user2_coordinates = (35.945, -119.000)  # Los Angeles

distance_km = haversine_distance(*user1_coordinates, *user2_coordinates)
print(f"Distance between the two users: {distance_km:.2f} kilometers")



Distance between the two users: 6.12 kilometers
